Based on a code by : https://www.kaggle.com/cstahl12/titanic-with-keras

### Load environment

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

Using TensorFlow backend.


In [2]:
raw_train = pd.read_csv('./train.csv', index_col = 0)
raw_train['is_test'] = 0
raw_test = pd.read_csv('./test.csv', index_col = 0)
raw_test['is_test'] = 1

In [3]:
all_data = pd.concat((raw_train,raw_test),axis=0)

### Functions to preprocess the data

In [4]:
def get_title_last_name(name):
    full_name = name.str.split(',', n=0, expand=True)
    last_name = full_name[0]
    titles = full_name[1].str.split('.', n=0, expand=True)
    titles = titles[0]
    return(titles)

def get_titles_from_names(df):
    df['Title'] = get_title_last_name(df['Name'])
    df = df.drop(['Name'], axis=1)
    return(df)

def get_dummy_cats(df):
    return(pd.get_dummies(df,columns=['Title','Pclass','Sex','Embarked','Cabin','Cabin_letter']))

def get_cabin_letter(df):
    df['Cabin'].fillna('Z', inplace=True)
    df['Cabin_letter'] = df['Cabin'].str[0]
    return(df)


def process_data(df):
    #preprocess titles, cabin, embarked
    df = get_titles_from_names(df)
    df['Embarked'].fillna('S', inplace=True)
    df=get_cabin_letter(df)
    
    #drop remaining features
    df = df.drop(['Ticket','Fare'], axis=1)
    
    #create dummy values for categorical features
    df = get_dummy_cats(df)
    
    return(df)

proc_data = process_data(all_data)
proc_train = proc_data[proc_data['is_test'] == 0]
proc_test = proc_data[proc_data['is_test'] == 1] 

In [5]:
proc_data.head()

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_Z,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,22.0,0,1,0.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,38.0,0,1,1.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,26.0,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,35.0,0,1,1.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,35.0,0,0,0.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


### Build Network to predict missing ages

In [6]:
for_age_train = proc_data.drop(['Survived', 'is_test'], axis = 1).dropna(axis=0)
X_train_age=for_age_train.drop('Age', axis=1)
y_train_age=for_age_train['Age']



In [7]:
#create model
tmodel = Sequential()
tmodel.add(Dense(input_dim=X_train_age.shape[1], units=128,
                kernel_initializer='normal',
                bias_initializer='zeros'))
tmodel.add(Activation('relu'))

for i in range(0,8):
    tmodel.add(Dense(units=64,kernel_initializer='he_normal',
                   bias_initializer='zeros'))
    tmodel.add(Activation('relu'))
    tmodel.add(Dropout(.25))
    
tmodel.add(Dense(units=1))
tmodel.add(Activation('linear'))

tmodel.compile(loss='mean_squared_error', optimizer='rmsprop')


In [8]:
tmodel.fit(X_train_age.values, y_train_age.values, epochs=600, verbose=2)

Epoch 1/600
0s - loss: 498.6623
Epoch 2/600
0s - loss: 248.4579
Epoch 3/600
0s - loss: 206.2251
Epoch 4/600
0s - loss: 210.0981
Epoch 5/600
0s - loss: 181.5337
Epoch 6/600
0s - loss: 187.8482
Epoch 7/600
0s - loss: 179.9447
Epoch 8/600
0s - loss: 178.1417
Epoch 9/600
0s - loss: 167.3470
Epoch 10/600
0s - loss: 158.8738
Epoch 11/600
0s - loss: 152.1813
Epoch 12/600
0s - loss: 164.9264
Epoch 13/600
0s - loss: 166.8291
Epoch 14/600
0s - loss: 139.5508
Epoch 15/600
0s - loss: 141.1015
Epoch 16/600
0s - loss: 142.4603
Epoch 17/600
0s - loss: 142.1889
Epoch 18/600
0s - loss: 133.1224
Epoch 19/600
0s - loss: 138.9748
Epoch 20/600
0s - loss: 131.5242
Epoch 21/600
0s - loss: 136.6832
Epoch 22/600
0s - loss: 124.9440
Epoch 23/600
0s - loss: 140.7357
Epoch 24/600
0s - loss: 122.4170
Epoch 25/600
0s - loss: 122.3572
Epoch 26/600
0s - loss: 122.8426
Epoch 27/600
0s - loss: 123.3324
Epoch 28/600
0s - loss: 118.5756
Epoch 29/600
0s - loss: 127.7602
Epoch 30/600
0s - loss: 125.2369
Epoch 31/600
0s - l

0s - loss: 86.5368
Epoch 252/600
0s - loss: 88.4900
Epoch 253/600
0s - loss: 80.5037
Epoch 254/600
0s - loss: 86.1983
Epoch 255/600
0s - loss: 88.8862
Epoch 256/600
0s - loss: 83.9776
Epoch 257/600
0s - loss: 84.1426
Epoch 258/600
0s - loss: 84.7873
Epoch 259/600
0s - loss: 82.2426
Epoch 260/600
0s - loss: 82.6211
Epoch 261/600
0s - loss: 86.1641
Epoch 262/600
0s - loss: 88.2724
Epoch 263/600
0s - loss: 82.2968
Epoch 264/600
0s - loss: 90.0412
Epoch 265/600
0s - loss: 87.1607
Epoch 266/600
0s - loss: 91.8177
Epoch 267/600
0s - loss: 87.5942
Epoch 268/600
0s - loss: 82.5395
Epoch 269/600
0s - loss: 91.6237
Epoch 270/600
0s - loss: 87.7858
Epoch 271/600
0s - loss: 84.0224
Epoch 272/600
0s - loss: 88.3253
Epoch 273/600
0s - loss: 86.5814
Epoch 274/600
0s - loss: 86.4019
Epoch 275/600
0s - loss: 88.3863
Epoch 276/600
0s - loss: 86.5091
Epoch 277/600
0s - loss: 84.5297
Epoch 278/600
0s - loss: 87.7985
Epoch 279/600
0s - loss: 85.5061
Epoch 280/600
0s - loss: 87.7376
Epoch 281/600
0s - loss:

0s - loss: 81.9510
Epoch 501/600
0s - loss: 81.6374
Epoch 502/600
0s - loss: 80.4863
Epoch 503/600
0s - loss: 79.4079
Epoch 504/600
0s - loss: 85.6625
Epoch 505/600
0s - loss: 80.9462
Epoch 506/600
0s - loss: 80.5977
Epoch 507/600
0s - loss: 80.0844
Epoch 508/600
0s - loss: 79.1295
Epoch 509/600
0s - loss: 79.2277
Epoch 510/600
0s - loss: 82.4577
Epoch 511/600
0s - loss: 79.1466
Epoch 512/600
0s - loss: 80.8159
Epoch 513/600
0s - loss: 81.1841
Epoch 514/600
0s - loss: 82.5775
Epoch 515/600
0s - loss: 80.2543
Epoch 516/600
0s - loss: 84.2056
Epoch 517/600
0s - loss: 79.9285
Epoch 518/600
0s - loss: 83.2330
Epoch 519/600
0s - loss: 81.1847
Epoch 520/600
0s - loss: 81.9329
Epoch 521/600
0s - loss: 81.4402
Epoch 522/600
0s - loss: 82.5496
Epoch 523/600
0s - loss: 82.8364
Epoch 524/600
0s - loss: 78.3613
Epoch 525/600
0s - loss: 82.6339
Epoch 526/600
0s - loss: 80.7686
Epoch 527/600
0s - loss: 81.2560
Epoch 528/600
0s - loss: 80.1395
Epoch 529/600
0s - loss: 78.9899
Epoch 530/600
0s - loss:

In [9]:
train_data = proc_train
train_data.loc[train_data['Age'].isnull()]

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_Z,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z
PassengerId,,,,,,,,,,,,,,,,,,,,,
6,NaN,0,0,0.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
18,NaN,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
20,NaN,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
27,NaN,0,0,0.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
29,NaN,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
30,NaN,0,0,0.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
32,NaN,0,1,1.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
33,NaN,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
37,NaN,0,0,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [10]:
to_pred = train_data.loc[train_data['Age'].isnull()].drop(['Age','Survived','is_test'],axis=1)
p = tmodel.predict(to_pred.values)
train_data['Age'].loc[train_data['Age'].isnull()] = p

/home/ikanez/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
test_data = proc_test
to_pred = test_data.loc[test_data['Age'].isnull()].drop(['Age','Survived','is_test'],axis=1)
p = tmodel.predict(to_pred.values)
test_data['Age'].loc[test_data['Age'].isnull()] = p

/home/ikanez/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
train_data.loc[train_data['Age'].isnull()]

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_Z,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z
PassengerId,,,,,,,,,,,,,,,,,,,,,


In [13]:
y = pd.get_dummies(train_data['Survived'])
y.head()

,0.0,1.0
PassengerId,,
1,1,0
2,0,1
3,0,1
4,0,1
5,1,0


In [14]:
X = train_data.drop(['Survived','is_test'], axis=1)

In [15]:
#create model

#titanic3
# model = Sequential()
# model.add(Dense(input_dim=X.shape[1],units=128, kernel_initializer='he_normal', bias_initializer='zeros'))
# model.add(Activation('relu'))

# for i in range(0,15):
#     model.add(Dense(units=256, kernel_initializer='he_normal', bias_initializer='zeros'))
#     model.add(Activation('relu'))
#     model.add(Dropout(.40))

# model.add(Dense(units=2))
# model.add(Activation('softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#titanic4 (0.7799)
model = Sequential()
model.add(Dense(input_dim=X.shape[1],units=128, kernel_initializer='he_normal', bias_initializer='zeros'))
model.add(Activation('relu'))

for i in range(0,8):
    model.add(Dense(units=384, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Activation('relu'))
    model.add(Dropout(.40))

model.add(Dense(units=2))
model.add(Activation('softmax'))


Train on 596 samples, validate on 295 samples
Epoch 1/300
596/596 [==============================] - 0s - loss: 2.4767 - acc: 0.5503 - val_loss: 0.6589 - val_acc: 0.6339
Epoch 2/300
596/596 [==============================] - 0s - loss: 1.2763 - acc: 0.5201 - val_loss: 0.6599 - val_acc: 0.6339
Epoch 3/300
596/596 [==============================] - 0s - loss: 0.8065 - acc: 0.5369 - val_loss: 0.6880 - val_acc: 0.6644
Epoch 4/300
596/596 [==============================] - 0s - loss: 0.7437 - acc: 0.5537 - val_loss: 0.6868 - val_acc: 0.6644
Epoch 5/300
596/596 [==============================] - 0s - loss: 0.7173 - acc: 0.5856 - val_loss: 0.6840 - val_acc: 0.6373
Epoch 6/300
596/596 [==============================] - 0s - loss: 0.7164 - acc: 0.5285 - val_loss: 0.6792 - val_acc: 0.6339
Epoch 7/300
596/596 [==============================] - 0s - loss: 0.6986 - acc: 0.5772 - val_loss: 0.6823 - val_acc: 0.6339
Epoch 8/300
596/596 [==============================] - 0s - loss: 0.6757 - acc: 0.5872

596/596 [==============================] - 0s - loss: 0.3445 - acc: 0.8775 - val_loss: 0.4050 - val_acc: 0.8407
Epoch 67/300
596/596 [==============================] - 0s - loss: 0.3542 - acc: 0.8691 - val_loss: 0.3908 - val_acc: 0.8339
Epoch 68/300
596/596 [==============================] - 0s - loss: 0.3537 - acc: 0.8792 - val_loss: 0.5159 - val_acc: 0.8373
Epoch 69/300
596/596 [==============================] - 0s - loss: 0.3548 - acc: 0.8826 - val_loss: 0.4224 - val_acc: 0.8441
Epoch 70/300
596/596 [==============================] - 0s - loss: 0.3452 - acc: 0.8792 - val_loss: 0.6094 - val_acc: 0.8203
Epoch 71/300
596/596 [==============================] - 0s - loss: 0.3481 - acc: 0.8842 - val_loss: 0.5000 - val_acc: 0.8203
Epoch 72/300
596/596 [==============================] - 0s - loss: 0.3424 - acc: 0.8842 - val_loss: 0.5234 - val_acc: 0.8169
Epoch 73/300
596/596 [==============================] - 0s - loss: 0.3367 - acc: 0.8826 - val_loss: 0.4235 - val_acc: 0.8542
Epoch 74/300


596/596 [==============================] - 0s - loss: 0.2974 - acc: 0.8859 - val_loss: 0.5564 - val_acc: 0.8271
Epoch 132/300
596/596 [==============================] - 0s - loss: 0.2905 - acc: 0.8960 - val_loss: 0.4977 - val_acc: 0.8271
Epoch 133/300
596/596 [==============================] - 0s - loss: 0.3041 - acc: 0.8926 - val_loss: 0.6548 - val_acc: 0.8237
Epoch 134/300
596/596 [==============================] - 0s - loss: 0.2970 - acc: 0.8859 - val_loss: 0.9780 - val_acc: 0.8203
Epoch 135/300
596/596 [==============================] - 0s - loss: 0.3039 - acc: 0.8826 - val_loss: 0.5284 - val_acc: 0.8203
Epoch 136/300
596/596 [==============================] - 0s - loss: 0.2857 - acc: 0.8926 - val_loss: 0.6758 - val_acc: 0.8068
Epoch 137/300
596/596 [==============================] - 0s - loss: 0.3439 - acc: 0.8943 - val_loss: 0.9761 - val_acc: 0.8373
Epoch 138/300
596/596 [==============================] - 0s - loss: 0.2995 - acc: 0.8893 - val_loss: 0.5791 - val_acc: 0.8203
Epoch 

596/596 [==============================] - 0s - loss: 0.3005 - acc: 0.8909 - val_loss: 0.6682 - val_acc: 0.8203
Epoch 197/300
596/596 [==============================] - 0s - loss: 0.2807 - acc: 0.8792 - val_loss: 0.7926 - val_acc: 0.8339
Epoch 198/300
596/596 [==============================] - 0s - loss: 0.2788 - acc: 0.8993 - val_loss: 0.6981 - val_acc: 0.8203
Epoch 199/300
596/596 [==============================] - 0s - loss: 0.2799 - acc: 0.8893 - val_loss: 0.8138 - val_acc: 0.8203
Epoch 200/300
596/596 [==============================] - 0s - loss: 0.2800 - acc: 0.8909 - val_loss: 0.8821 - val_acc: 0.8203
Epoch 201/300
596/596 [==============================] - 0s - loss: 0.2777 - acc: 0.8977 - val_loss: 0.8291 - val_acc: 0.8136
Epoch 202/300
596/596 [==============================] - 0s - loss: 0.2850 - acc: 0.8960 - val_loss: 0.7487 - val_acc: 0.8305
Epoch 203/300
596/596 [==============================] - 0s - loss: 0.2817 - acc: 0.8993 - val_loss: 0.8145 - val_acc: 0.8271
Epoch 

596/596 [==============================] - 0s - loss: 0.3052 - acc: 0.8859 - val_loss: 0.9005 - val_acc: 0.8068
Epoch 262/300
596/596 [==============================] - 0s - loss: 0.2875 - acc: 0.8926 - val_loss: 1.1002 - val_acc: 0.8339
Epoch 263/300
596/596 [==============================] - 0s - loss: 0.2939 - acc: 0.8893 - val_loss: 1.0078 - val_acc: 0.8034
Epoch 264/300
596/596 [==============================] - 0s - loss: 0.2868 - acc: 0.8993 - val_loss: 0.8630 - val_acc: 0.8203
Epoch 265/300
596/596 [==============================] - 0s - loss: 0.2755 - acc: 0.9010 - val_loss: 1.1327 - val_acc: 0.8169
Epoch 266/300
596/596 [==============================] - 0s - loss: 0.2765 - acc: 0.8977 - val_loss: 1.0760 - val_acc: 0.8373
Epoch 267/300
596/596 [==============================] - 0s - loss: 0.3058 - acc: 0.8859 - val_loss: 1.0277 - val_acc: 0.8169
Epoch 268/300
596/596 [==============================] - 0s - loss: 0.2787 - acc: 0.8960 - val_loss: 1.0629 - val_acc: 0.8373
Epoch 

In [16]:
model.fit(X.values, y.values, epochs=300,validation_split=0.33, batch_size=20, verbose=1, shuffle=True)

Train on 596 samples, validate on 295 samples
Epoch 1/300
596/596 [==============================] - 0s - loss: 0.2623 - acc: 0.8977 - val_loss: 0.7765 - val_acc: 0.8203
Epoch 2/300
596/596 [==============================] - 0s - loss: 0.2708 - acc: 0.8960 - val_loss: 0.8909 - val_acc: 0.8339
Epoch 3/300
596/596 [==============================] - 0s - loss: 0.2967 - acc: 0.8943 - val_loss: 1.0707 - val_acc: 0.8305
Epoch 4/300
596/596 [==============================] - 0s - loss: 0.2697 - acc: 0.8960 - val_loss: 1.0217 - val_acc: 0.8237
Epoch 5/300
596/596 [==============================] - 0s - loss: 0.2813 - acc: 0.8977 - val_loss: 0.8487 - val_acc: 0.8339
Epoch 6/300
596/596 [==============================] - 0s - loss: 0.2867 - acc: 0.8842 - val_loss: 1.0852 - val_acc: 0.8237
Epoch 7/300
596/596 [==============================] - 0s - loss: 0.2942 - acc: 0.8926 - val_loss: 0.9510 - val_acc: 0.8271
Epoch 8/300
596/596 [==============================] - 0s - loss: 0.2707 - acc: 0.8960

596/596 [==============================] - 0s - loss: 0.2542 - acc: 0.9044 - val_loss: 1.2504 - val_acc: 0.8237
Epoch 67/300
596/596 [==============================] - 0s - loss: 0.3099 - acc: 0.8943 - val_loss: 0.9728 - val_acc: 0.8271
Epoch 68/300
596/596 [==============================] - 0s - loss: 0.2759 - acc: 0.8859 - val_loss: 1.0761 - val_acc: 0.8271
Epoch 69/300
596/596 [==============================] - 0s - loss: 0.2856 - acc: 0.8909 - val_loss: 0.9161 - val_acc: 0.8203
Epoch 70/300
596/596 [==============================] - 0s - loss: 0.2721 - acc: 0.8960 - val_loss: 1.0041 - val_acc: 0.8237
Epoch 71/300
596/596 [==============================] - 0s - loss: 0.2658 - acc: 0.9027 - val_loss: 1.0298 - val_acc: 0.8203
Epoch 72/300
596/596 [==============================] - 0s - loss: 0.2698 - acc: 0.8977 - val_loss: 1.1470 - val_acc: 0.8339
Epoch 73/300
596/596 [==============================] - 0s - loss: 0.2582 - acc: 0.9044 - val_loss: 1.5083 - val_acc: 0.8237
Epoch 74/300


596/596 [==============================] - 0s - loss: 0.2856 - acc: 0.8943 - val_loss: 0.8084 - val_acc: 0.8339
Epoch 132/300
596/596 [==============================] - 0s - loss: 0.2873 - acc: 0.8909 - val_loss: 0.9022 - val_acc: 0.8339
Epoch 133/300
596/596 [==============================] - 0s - loss: 0.2715 - acc: 0.8993 - val_loss: 0.8062 - val_acc: 0.8305
Epoch 134/300
596/596 [==============================] - 0s - loss: 0.2881 - acc: 0.8960 - val_loss: 0.7384 - val_acc: 0.8305
Epoch 135/300
596/596 [==============================] - 0s - loss: 0.2796 - acc: 0.8893 - val_loss: 0.8084 - val_acc: 0.8373
Epoch 136/300
596/596 [==============================] - 0s - loss: 0.2706 - acc: 0.8977 - val_loss: 0.9690 - val_acc: 0.8271
Epoch 137/300
596/596 [==============================] - 0s - loss: 0.2674 - acc: 0.8993 - val_loss: 1.0142 - val_acc: 0.8305
Epoch 138/300
596/596 [==============================] - 0s - loss: 0.2713 - acc: 0.8977 - val_loss: 1.1029 - val_acc: 0.8339
Epoch 

596/596 [==============================] - 0s - loss: 0.2668 - acc: 0.8960 - val_loss: 1.6377 - val_acc: 0.8237
Epoch 197/300
596/596 [==============================] - 0s - loss: 0.4451 - acc: 0.8876 - val_loss: 1.3815 - val_acc: 0.8000
Epoch 198/300
596/596 [==============================] - 0s - loss: 0.4346 - acc: 0.8658 - val_loss: 0.7508 - val_acc: 0.8339
Epoch 199/300
596/596 [==============================] - 0s - loss: 0.3718 - acc: 0.8691 - val_loss: 0.7622 - val_acc: 0.8169
Epoch 200/300
596/596 [==============================] - 0s - loss: 0.3025 - acc: 0.8826 - val_loss: 0.8992 - val_acc: 0.8441
Epoch 201/300
596/596 [==============================] - 0s - loss: 0.3285 - acc: 0.8876 - val_loss: 0.9040 - val_acc: 0.8136
Epoch 202/300
596/596 [==============================] - 0s - loss: 0.3324 - acc: 0.8809 - val_loss: 0.9663 - val_acc: 0.8000
Epoch 203/300
596/596 [==============================] - 0s - loss: 0.2977 - acc: 0.8859 - val_loss: 0.9605 - val_acc: 0.8034
Epoch 

596/596 [==============================] - 0s - loss: 0.2928 - acc: 0.8876 - val_loss: 1.3517 - val_acc: 0.8136
Epoch 262/300
596/596 [==============================] - 0s - loss: 0.2815 - acc: 0.8977 - val_loss: 1.4037 - val_acc: 0.8000
Epoch 263/300
596/596 [==============================] - 0s - loss: 0.3206 - acc: 0.8842 - val_loss: 1.4085 - val_acc: 0.8000
Epoch 264/300
596/596 [==============================] - 0s - loss: 0.3046 - acc: 0.8893 - val_loss: 1.4812 - val_acc: 0.8068
Epoch 265/300
596/596 [==============================] - 0s - loss: 0.2830 - acc: 0.8960 - val_loss: 1.2685 - val_acc: 0.8102
Epoch 266/300
596/596 [==============================] - 0s - loss: 0.2765 - acc: 0.8977 - val_loss: 1.2535 - val_acc: 0.8271
Epoch 267/300
596/596 [==============================] - 0s - loss: 0.2801 - acc: 0.8993 - val_loss: 1.1442 - val_acc: 0.8136
Epoch 268/300
596/596 [==============================] - 0s - loss: 0.2729 - acc: 0.8859 - val_loss: 1.4435 - val_acc: 0.8169
Epoch 

In [17]:
test_data.columns

Index(['Age', 'Parch', 'SibSp', 'Survived', 'is_test', 'Title_ Capt',
       'Title_ Col', 'Title_ Don', 'Title_ Dona', 'Title_ Dr',
       ...
       'Cabin_Z', 'Cabin_letter_A', 'Cabin_letter_B', 'Cabin_letter_C',
       'Cabin_letter_D', 'Cabin_letter_E', 'Cabin_letter_F', 'Cabin_letter_G',
       'Cabin_letter_T', 'Cabin_letter_Z'],
      dtype='object', length=227)

In [18]:
p_survived = model.predict_classes(test_data.drop(['Survived','is_test'],axis = 1).values)




 32/418 [=>............................] - ETA: 0s

In [19]:
submission = pd.DataFrame()
submission['PassengerId'] = test_data.index
submission['Survived'] = p_survived

In [20]:
submission.shape

(418, 2)

In [21]:
submission.to_csv('titanic_keras2.csv', index=False)